Copyright (c) European Space Agency, 2025.

<small><em>This file is subject to the terms and conditions defined in file 'LICENCE.txt', which is part of this source code package. No part of the package, including this file, may be copied, modified, propagated, or distributed except according to the terms contained in the file ‘LICENCE.txt’.</em></small>
***

# <font color=green> Case Study: X-ray Binary Vela X-1  </font>
***
## Introduction
This thread serves as a comprehensive, step-by-step guide illustrating a mini science project leveraging XMM-Newton data. The focal point is the retrieval and processing of data associated with the high-mass X-ray binary Vela X-1, which is composed of a neutron star. Subsequently, the derived spectrum can undergo in depth analysis using pyXspec in accordance with the user's wishes, where if wanted they can apply various models to draw insightful conclusions from the observational data.

Moreover, this thread exemplifies the practical utilization of the XMM-SAS Datalab for research purposes. This platform facilitates a streamlined experience, enabling users to promptly obtain data in their preferred format and conduct comprehensive analyses, all within a unified space. Importantly, this eliminates the need for additional package installations or the inconvenience of navigating between disparate platforms.

On this thread we demonstrate applications of methods such as SAS start-up, reprocessing the ODF to generate concatenated EPIC event lists, extracting ligthcurves, checking for pile-up, and extracting a spectrum which can be used for scientific analysis.

## Background Information

Vela X-1 is a prominent X-ray binary system located in the Vela constellation. Discovered in the early 1970s by the Uhuru X-ray satellite, however it has been observed by nearly all X-ray observatories even multiple times. This binary system features a pulsar—a rapidly rotating neutron star—as its compact object. The companion star is a massive B-type supergiant, and the system is characterized by the transfer of mass from the companion to the neutron star. As the material from the donor star accretes onto the neutron star's surface, it emits intense X-ray radiation, making Vela X-1 detectable in X-ray observations, while also providing a rich spectrum with various emission lines. The high rate of rotation of the neutron star, combined with the mass transfer process, results in the system's dynamic and variable behavior. Vela X-1's study has contributed valuable insights into the physics of accretion processes, neutron star characteristics, and the evolution of binary systems involving compact objects.

The particular observation we chose is with ObsID of 0841890201. Total exposures are five with 3 EPIC, 0 OM, and 2 RGS. This is a very bright source, hence EPIC-pn exposure is in timing mode, and when processing adjustments such as pile-up correction will be necessary.


## Suggested Further Reading
* [Revisiting the archetypical wind accretor Vela X-1 in depth](https://doi.org/10.1051/0004-6361/202040272)
* [PyXspec](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html)
* [Observing the onset of the accretion wake in Vela X-1](https://ui.adsabs.harvard.edu/link_gateway/2023A&A...674A.147D/doi:10.48550/arXiv.2303.09631)
* [A NICER viewing angle on the accretion stream of Vela X-1](https://ui.adsabs.harvard.edu/link_gateway/2023ApJ...950..170R/doi:10.48550/arXiv.2302.10953)
* [Continuum, cyclotron line, and absorption variability in the high-mass X-ray binary Vela X-1](https://ui.adsabs.harvard.edu/abs/2022A%26A...660A..19D/abstract)

## Further Credits
Special thanks and recognition to [Camille Diez](https://www.camillediez.com/) for sharing her research method, knowledge, and help.

### *SAS version used v21*

***

Import the following libraries listed below:

In [ ]:
from pysas.wrapper import Wrapper as w
import os.path
from os import path
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from matplotlib.colors import LogNorm
from matplotlib.ticker import ScalarFormatter  
from xspec import *
import re
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from tools.js9helper import *
from tools.plotLC import *
import shutil
from IPython.display import display, Image
from tools.xspecplot import *

* ### SAS start-up

Use the 'expanduser' function from the 'os.path' module to get the home directory of the current user. The variable 'home' now will contain the path to the home directory of the current user:

In [ ]:
home = os.path.expanduser('~')

In [ ]:
notebook_dir = os.getcwd()
print(f"Current working directory: {notebook_dir}")

Create the working dircetory where all the files produced along the process can be stroed and accessed easily below. Make sure to save your files inside the `my_workspace` folder for permanent storage, as everything stored outside of this folder will be erased every time the Datalab is closed.

In [ ]:
wdir=f'{home}/VelaX1-data'

#Check if the working directory exists, and create it if it doesn't
if not os.path.exists(wdir):
    os.makedirs(wdir)

#Change to working directory
os.chdir(wdir)

print(f"Current working directory: {os.getcwd()}")

Define the path to the CCF files provided as a data volume by Datalabs. Hence the user des not need to download any CCF files nor worry about updates. For more information on how to add them in your Datalabs data volume, please refer to the general guidelines on the ESA Datalabs Help section.

In [ ]:
# Type the path you have used to mount the CCF directory. 
ccf_paths = ['/data/user/pub', '/data/pub']

In [ ]:
for user_ccfpath in ccf_paths:
    ccf_path = f'{home}{user_ccfpath}'
    if os.path.isdir(ccf_path):
        os.environ['SAS_CCFPATH'] = ccf_path
        print("Path to the XMM-Newton CCFs: " + ccf_path + "\n")
        break
else:
    raise FileNotFoundError("Cannot locate the specified CCF paths, please check your data volume.")

Before starting, it is a good practice to sun `sasver` to check which SAS variables are defined.

In [ ]:
inargs = []
t = w('sasver', inargs)
t.run()

Next, we execute the command `startsas`. The task downloads the ODFs of the odfid the user defines in the working directory defined earlier. Please refer to the [XMM-Newton Science Archive](https://www.cosmos.esa.int/web/xmm-newton/xsa) to search through the catalogue for different observation IDs. We will be working with ODF 0841890201.

In [ ]:
t = w('startsas',['odfid=0841890201'])
t.run()

* ### Running `epproc`

In [ ]:
inargs = [f'sas_ccf={wdir}/ccf.cif', f'sas_odf={wdir}/3553_0841890201_SCX00000SUM.SAS', f'workdir={wdir}']

w('startsas', inargs).run()

We will now apply `epproc` for this observation that has been taken in timing mode. Timing mode can be used for very bright sources (such as this observation of Vela X-1), which require a shorter readout time. In timing mode the arrival times of individual X-ray photons are recorded with high precision. Rather than focusing on spatial information, this mode emphasizes the temporal aspects of X-ray events, making it ideal for studying time-dependent phenomena. Timing mode can allow us to extract detailed information about periodicities, rotational periods, and other temporal characteristics, providing insights into the dynamic behavior of Vela X-1. The default intrinsic timing mode calibration settings when running the `epproc` task are shown below:
```
TIMING: withrdpha=“Y”, withxrlcorrection=“Y”, runepreject=“Y”, runepfast=“N”
```
One can change the default calibration settings by adding `withdefaultcal=“N”` as argument of the `epproc` task and set the aforementioned calibration parameters to their own preference. If further information is needed please refer to the thread *How to Reprocess ODFS to Generate Calibrated and Concatenated EPIC Event Lists* and to the Appendix of [Diez et al. (2023)](https://arxiv.org/pdf/2303.09631.pdf) for issues with default calibration settings when in timing mode. For this exercise, we apply the same calibration corrections as in [Diez et al. (2023)](https://arxiv.org/pdf/2303.09631.pdf).

In [ ]:
# SAS Command
cmd    = 'epproc'  # SAS task to be executed

# Arguments of SAS Command, leave empty if you want to run with default calibration settings for timing mode
inargs = ['withdefaultcal=no', 'withrdpha=no', 'runepreject=yes', 'withxrlcorrection=yes', 'runepfast=yes']        #remove arguments for default cal
# comma separated arguments for SAS task

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
print("Running epproc ..... \n")

# Check if epproc has already run. If it has, do not run again 
exists = 0
pnevt_list = []
for root, dirs, files in os.walk("."):
    for filename in files:
        if (filename.find('EPN') != -1) and filename.endswith('TimingEvts.ds'):
            pnevt_list.append(filename)
            exists = 1
if exists:
    print(" > " + str(len(pnevt_list)) + " EPIC-pn event list found. Not running epproc again.\n")
    for x in pnevt_list:
        print("    " + x + "\n")
    print("..... OK")
else:
    w(cmd,inargs).run()      # <<<<< Execute SAS task
    exists = 0
    pnevt_list = []
    for root, dirs, files in os.walk("."):
        for filename in files:
            if (filename.find('EPN') != -1) and filename.endswith('TimingEvts.ds'):
                pnevt_list.append(filename)
                exists = 1
    if exists:
        print(" > " + str(len(pnevt_list)) + " EPIC-pn event list found after running epproc.\n")
        for x in pnevt_list:
            print("    " + x + "\n")
        print("..... OK")
    else:
        print("Something has gone wrong with epproc. I cant find any event list files after running. \n")

* ### Do I need to apply filtering for flaring particle background?

You do not always need to apply filtering for flaring particle background. In this scenario the source brightness is high enough to not require any filtering, because the flux of the object would be in every case larger than the possible flaring particle background. However, let us demonstrate this is indeed the case for demonstration purposes by inspecting the source and flaring particle background lightcurves.

We define our event file produced n the previous step:

In [ ]:
eventfile = wdir+'/3553_0841890201_EPN_S003_TimingEvts.ds'

In [ ]:
# Define a SAS filter expression to derive a background rate cut

pn_pattern   = 0        # pattern selection
pn_pi_min    = 10000.   # Low energy range eV
pn_pi_max    = 12000.   # High energy range eV
pn_threshold = 0.4      # cts/sec (only used here for display purposes)

out_LCFile   = wdir+'/PN_bkg_nrjthreshold.fits'  # Name of the output BKG lightcurve

The argument values can be altered depending on the user, these are only display numbers. After observing the produced light curve, the user can either come back and change values above and rerun the kernels or just copy the same kernels with the new values.

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'#XMMEA_EP&&(PI>={pn_pi_min}&&PI<={pn_pi_max})&&(PATTERN=={pn_pattern})'  # event filter expression
inargs     = [f'table={eventfile}','withrateset=Y',f'rateset={out_LCFile}','maketimecolumn=Y','timebinsize=100','makeratecolumn=Y',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
# Execute SAS task with parameters

w(cmd, inargs).run()

In [ ]:
# Plot BKG lightcurve
name= ["background light curve"]
file_paths = [out_LCFile]

# Call the function to plot light curves
plotLC(file_paths, name, threshold = None)


In [ ]:
pn_pi_min    = 150   
out_file = wdir + '/PN_sr_Rthreshold.fits'

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'#XMMEA_EP&&(PI>={pn_pi_min})'  # event filter expression
inargs     = [f'table={eventfile}','withrateset=Y',f'rateset={out_file}','maketimecolumn=Y','timebinsize=283.44','makeratecolumn=Y',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
# Execute SAS task with parameters

w(cmd, inargs).run()

In [ ]:
name= [" "]
file_paths = [out_file]

# Call the function to plot light curves
plotLC(file_paths, name, threshold = None)


* ### Barycentric correction

Here we apply a barycentric correction, necessary
for high resolved timing analysis. 

After inspecting the light curve, the user can choose to remove the threshold value altogether if they see no need for it. This is particularly applicable in scenarios where the source is bright enough, rendering particle background filtering unnecessary. Hence we can run `evselect` in the following conditions:

In [ ]:
pn_pi_min    = 150   
out_file = wdir + '/PN_clean_evt.fits'

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'#XMMEA_EP&&(PI>={pn_pi_min})'  # event filter expression
inargs     = [f'table={eventfile}','withfilteredset=Y',f'filteredset={out_file}','destruct=Y','keepfilteroutput=T',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
# Execute SAS task with parameters

w(cmd, inargs).run()

Next we shall run the task `barycen`. The `barycen` task is designed to convert times from the local satellite frame to Barycentric Dynamical Time (TDB) using a specified table in a dataset containing times in XMM MET (Mission Elapsed Time) format. The task checks if barycentric conversion has been performed, exits if true, and proceeds to correct time tags in the specified column. Additionally, it converts interval start and stop times to TDB if a Good Time Interval extension exists and corrects time tags in EXPOSU tables if the relevant parameter is enabled. The task updates TIMEREF, TSTART, TSTOP, and TELAPSE attributes where necessary.

In [ ]:
# Use the 'copy' function from the 'shutil' module to copy the file 'PN_clean_evt.fits'
# to a new file named 'PN_clean_evt_nobarycen_cor.fits'.
shutil.copy('PN_clean_evt.fits', 'PN_clean_evt_nobarycen_cor.fits')

In [ ]:
table = wdir + "/PN_clean_evt.fits"

In [ ]:
# SAS Command
cmd        = "barycen" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = ['withtable=true', f'table={table}:EVENTS']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
#|%%capture
# Execute SAS task with parameters

w(cmd, inargs).run()

Now we have created filtered PN event list files with and without barycen. This barycentric correction will be necessary
for high resolved timing analysis later on.

* ### Extracting the average light curve

Let us produce an image from our event file. 

In [ ]:
# Define some parameters to produce the image and the name of the output file

xbin=1     # xbin size
ybin=1    # ybin size
xcoord='RAWX'  # coordinate system
ycoord='RAWY'  # coordinate system

out_IMFile   = wdir+'/PNimage.fits'  # Name of the output Image file 

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = [f'table={table}','imagebinning=binSize',f'imageset={out_IMFile}','withimageset=yes',f'xcolumn={xcoord}',f'ycolumn={ycoord}',f'ximagebinsize={xbin}',f'yimagebinsize={ybin}']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In the image file the RAWY column is time, as the image produced was in timing mode. The source should be visible as a long bright line. Now we need to define the RARX1 and RAWX2 coordinate range for the strip of bright line which is the source, for this we can use js9 to get the coordination of the pixels.

Below we load the image produced in js9 using a custom function, so we can determine our RAWX1 and RAWX2 coordinates.

In [ ]:
my_js9 = jpyjs9.JS9(width=600, height=700)

In [ ]:
visualise(my_js9, out_IMFile)

The values chosen in this case are taken from the paper of [Diez et al. (2023)](https://arxiv.org/pdf/2303.09631.pdf), where they study the same observation. However, the user can feel free to play around and experiment with their own values.

In [ ]:
rawX1src = 32 #RAWX1 of the source rectangle region (in pixels)
rawX2src = 44 #RAWX2 of the source rectangle region (in pixels)

In [ ]:
# Extract the source region lightcurve

# Define some parameters for filtering the event file and define the lightcurve binning

q_flag       = "#XMMEA_EP" # Quality flag for EPIC pn
n_pattern    = 4           # Pattern selection
pn_pi_min    = 500.        # Low energy range eV
pn_pi_max    = 10000.      # High energy range eV
lc_bin       = 283         # Lightcurve bin in secs

# Define the output ligthcurve file name

in_LCSRCFile = wdir+'/PN_source_lightcurve_raw_bin283sec.lc'   # Name of the output source lightcurve

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'{q_flag}&&(PATTERN<={n_pattern})&&(RAWX in [{rawX1src}:{rawX2src}])&&(PI in [{pn_pi_min}:{pn_pi_max}])'  # event filter expression
inargs     = [f'table={table}','energycolumn=PI','withrateset=yes',f'rateset={in_LCSRCFile}',
              f'timebinsize={lc_bin}','maketimecolumn=yes','makeratecolumn=yes',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

After extration we can plot the lightcurve of the selected source region:

In [ ]:
name= ["Source with 283 bin in sec"]
file_paths = [in_LCSRCFile]
plotVelaX1LC(file_paths, name, threshold=None, figname="src_lightcurve.png")

Next moving onto extracting the lightcurve of the background region, where again pixel coordinate values have been taken from [Diez et al. (2023)](https://arxiv.org/pdf/2303.09631.pdf):

In [ ]:
rawX1bkg = 3 #RAWX1 of the bkg rectangle region (in pixels)
rawX2bkg = 5 #RAWX2 of the bkg rectangle region (in pixels)

In [ ]:
# Extract the background region lightcurve

# Define some parameters for filtering the event file and define the lightcurve binning

q_flag       = "#XMMEA_EP" # Quality flag for EPIC pn
n_pattern    = 4           # Pattern selection
pn_pi_min    = 500.        # Low energy range eV
pn_pi_max    = 10000.      # High energy range eV
lc_bin       = 283         # Lightcurve bin in secs

# Define the output ligthcurve file name

in_LCBKGFile = wdir+'/PN_lightcurve_background_raw_bin283sec.lc'   # Name of the output source lightcurve

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'{q_flag}&&(PATTERN<={n_pattern})&&(RAWX in [{rawX1bkg}:{rawX2bkg}])&&(PI in [{pn_pi_min}:{pn_pi_max}])'  # event filter expression
inargs     = [f'table={table}','energycolumn=PI','withrateset=yes',f'rateset={in_LCBKGFile}',
              f'timebinsize={lc_bin}','maketimecolumn=yes','makeratecolumn=yes',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
name= ["Background with 283 bins in sec"]
file_paths = [in_LCBKGFile]
plotVelaX1LC(file_paths, name, threshold=None, figname="bkg_lightcurve.png")

For background extraction we can run the command `epiclccorr` to perform correction of the lightcurve. Hence the command requires as input both light curves (which are used to establish the binning of the final corrected background subtracted light curve) and the event file.

In [ ]:
# Define the output corrected ligthcurve file name

in_LCFile = wdir+'/PN_lccorr_bin283sec.lc'   # Name of the output corrected lightcurve

In [ ]:
# Correct the light curve with the SAS task epiclccorr

# SAS Command
cmd        = "epiclccorr" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = [f'eventlist={table}',f'srctslist={in_LCSRCFile}',f'outset={in_LCFile}',
              f'bkgtslist={in_LCBKGFile}','withbkgset=yes','applyabsolutecorrections=yes']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

Let us visualize all the lightcurves produced using matplotlib lightcurve plotting function define by us through the plotLC python file, where we can bin some of the data for a cleaner look:

In [ ]:
# Inspect the light curves
fileNames = [in_LCFile, in_LCSRCFile]
names = ['Corrected', 'Source']

plotVelaX1LC(fileNames, names, threshold=None, figname="combined_lightcurvenothreshold.png")

* ### Checking for Pile-Up

"Pile-up" happens when the detector registers multiple X-ray photons within a single readout frame as a single event. This occurs when the X-ray flux is so high that the detector cannot distinguish individual photons, and their signals overlap or pile up.

In XMM-Newton, each incoming X-ray photon is converted into an electronic signal. However, if multiple photons arrive at the detector in rapid succession, their signals may overlap, leading to an inaccurate representation of the detected X-ray events. This phenomenon can distort the measured X-ray spectra and impact the accuracy of scientific observations.

This step is usually advised to be applied before extracting the light curves as done on the previous step, however as the source we look at is immensely bright, the pile-up effect will not have an significant impact on the output light curve. However, we apply it here as before the extraction of the spectrum it is important, otherwise it could lead to false output with the spectrum having higher flux at high energies.

In [ ]:
# Define some parameters for filtering the event file
rawX1src = 32
rawX2src = 44
pn_pi_min    = 550.        # Low energy range eV
pn_pi_max    = 10000.      # High energy range eV

# Define the output file name

filtered_output = wdir+'/PN_filtered_'+str(rawX1src)+'-'+str(rawX2src)+'.evt' 

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'(RAWX in [{rawX1src}:{rawX2src}])&&(PI in [{pn_pi_min}:{pn_pi_max}])'  # event filter expression
inargs     = [f'table={table}','withfilteredset=yes',f'filteredset={filtered_output}',
              'keepfilteroutput=yes',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

We can use the SAS command `epatplot` to plot EPIC pn and MOS event pattern statistics, which should help us see if pile-up has occured. For further in depth information the user is advised to look at the thread *How to Evaluate and test Pile-Up in an EPIC Source*.

In [ ]:
%%capture
w('epatplot',[f'set={filtered_output}']).run()

We can check the image produced in our working directory. It is a PDF file named: PN_filtered_32-44_pat.pdf .

As we can see the double events are higher than theory line and single ones are lower for our observation, thereby indicating the need for applying a pile-up correction, which we apply below:

In [ ]:
# Define some parameters for filtering the event file 
rawX3src = 36
rawX4src = 40
pn_pi_min    = 550.        # Low energy range eV
pn_pi_max    = 10000.      # High energy range eV

# Define the output file name

filtered_output = wdir+'/PN_filtered_annulus_'+str(rawX1src)+'-'+str(rawX3src)+'_'+str(rawX4src)+'-'+str(rawX2src)+'.evt'   # Name of the output 

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'(RAWX in [{rawX1src}:{rawX3src}] || RAWX in [{rawX4src}:{rawX2src}])&&(PI in [{pn_pi_min}:{pn_pi_max}])'  # event filter expression
inargs     = [f'table={table}','withfilteredset=yes',f'filteredset={filtered_output}',
              'keepfilteroutput=yes',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
%%capture
w('epatplot',[f'set={filtered_output}']).run()

Check the pdf produced in the working directory named PN_filtered_annulus_32-36_40-44_pat.pdf. Now we can see that after the correction the theory and observation match and we have accounted for pile-up, therefore we can move onto extracting the spectrum of the source safely.

* ### Spectrum Extraction

#### Time Averaged Spectrum

No we can produce a spectrum for the source region, by avoiding pile-up regions:

In [ ]:
rawX1src= 32
rawX2src = 44

In [ ]:
#core of the psf coordinates:
rawX3src = 36
rawX4src = 40
spectrumset= wdir+ '/PN_source_spectrum_raw_'+str(rawX1src)+'-'+str(rawX3src)+'_'+str(rawX4src)+'-'+str(rawX2src)+'.fits'

In [ ]:
# SAS Command
cmd        = "evselect" # SAS task to be executed                  

# Arguments of SAS Command
expression = f'(FLAG==0) && (PATTERN<=4) && (RAWX in [{rawX1src}:{rawX3src}] || RAWX in [{rawX4src}:{rawX2src}])'  # event filter expression
inargs     = [f'table={table}','withspectrumset=yes',f'spectrumset={spectrumset}',
              'energycolumn=PI', 'spectralbinsize=5', 'withspecranges=yes', 'specchannelmin=0', 'specchannelmax=20479',f'expression={expression}']

print("   Filter expression to use: "+expression+" \n")
print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
# SAS Command
cmd        = "backscale" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = [f'spectrumset={spectrumset}',f'badpixlocation={table}']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
# Define some parameters for rmfgen

# Define the output redistribution matrix file name

in_RESPFile = wdir+'/PN_'+str(rawX1src)+'-'+str(rawX3src)+'_'+str(rawX4src)+'-'+str(rawX2src)+'.rmf'   # Name of the output redistribution

In [ ]:
# SAS Command
cmd        = "rmfgen" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = [f'spectrumset={spectrumset}',f'rmfset={in_RESPFile}']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
# Define some parameters for arfgen

# Define the output ancillary file name

in_ARFFile = wdir+'/PN_'+str(rawX1src)+'-'+str(rawX3src)+'_'+str(rawX4src)+'-'+str(rawX2src)+'.arf'   # Name of the output ancillary

In [ ]:
# SAS Command
cmd        = "arfgen" # SAS task to be executed                  

print("   Checking for Response File ..... \n")
# Check if RESP file is available.
if os.path.isfile(in_RESPFile):
    print ("File "+in_RESPFile+" exists. \n")
else:
    print ("File "+in_RESPFile+" does not exist, please check. \n")

# Arguments of SAS Command
inargs     = [f'spectrumset={spectrumset}',f'arfset={in_ARFFile}',
              'withrmfset=yes',f'rmfset={in_RESPFile}',f'badpixlocation={table}','detmaptype=psf', 'applyabsfluxcorr=yes']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

In [ ]:
# Define some parameters for specgruop

in_GRPFile = wdir+'/PN_spectrum_grp_'+str(rawX1src)+'-'+str(rawX3src)+'_'+str(rawX4src)+'-'+str(rawX2src)+'.fits'   # Name of the output specgruop

In [ ]:
# SAS Command
cmd        = "specgroup" # SAS task to be executed                  

# Arguments of SAS Command
inargs     = [f'spectrumset={spectrumset}','mincounts=25','oversample=3',
              f'rmfset={in_RESPFile}',f'arfset={in_ARFFile}',
              f'groupedset={in_GRPFile}']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
%%capture
w(cmd, inargs).run()

Now that we have acquired the final spectrum, we can load the file using pyXspec:

In [ ]:
# Before you start, clear all data and models

AllModels.clear()  # Clear all models
AllData.clear()    # Clear all data

In [ ]:
# Load the group spectral file

# The group file already contains in the header the names of all the necessary files needed by xspec
# (source and background spectra, response and ancillery files)

data1 = Spectrum(in_GRPFile)  # load spectra groupped file

AllData.show()                # inspect loaded data

In [ ]:
# Set the plot device to /null to suppress graphical output

Plot.device = "/null"

In [ ]:
# First inspection of the spectrum

Plot.xAxis = "keV"        # set X axis to energy units
Plot.xLog  = True         # log scale
Plot.yLog  = True         # log scale
Plot("data")   # plot source and background spectra

In [ ]:
# Extract data from the XSPEC plot
energies = Plot.x(1)    # X-axis values
edeltas = Plot.xErr(1)  # X-axis error values
rates = Plot.y(1)       # Y-axis values
errors = Plot.yErr(1)   # Y-axis error values
labels = Plot.labels()

In [ ]:
# Customize the plot appearance
plt.xscale('log')
plt.yscale('log')
plt.xlabel(labels[0])
plt.ylabel(labels[1])
plt.title('Averaged Spectrum Vela X-1: Datalabs')
plt.tick_params(axis='both', which='both', direction='in', top=True, right=True)
plt.tick_params(axis='both', which='major', labelsize=13, length=10, width=1)
plt.tick_params(axis='both', which='minor', length=5, width=1)
plt.xticks([0.5, 1, 2, 5, 10], labels=[0.5, 1, 2, 5, 10])

# Plot your spectrum with error bars
plt.errorbar(energies,rates,xerr=edeltas,yerr=errors,fmt='.')

plt.savefig(wdir+"/averaged-spec-datalabs.png", dpi=300)

Above we have visualised an average spectrum, and to fit model the user needs to look at time resolved spectroscopy. In this thread we will not go into the specifics of model fitting for the spectrum.

#### Spectra for three distinct observation phases (see Fig. 3 from [Diez et al. (2023)](https://arxiv.org/pdf/2303.09631.pdf))

The phases are characterised by their hardness ratio as observed in [Diez et al. (2022)](https://ui.adsabs.harvard.edu/abs/2022A%26A...660A..19D/abstract):
    
* Stable hardness ratio at: $ 0.37 \lesssim Orbital Phase \lesssim 0.44 $
* Phase 2: $ 0.44 \lesssim Orbital Phase \lesssim 0.46 $
* Rise in the hardness ratio at: $ 0.46 \lesssim Orbital Phase \lesssim 0.51 $

Instead of running the detailed spectrum extraction steps showcased above, we have put the same tasks in a Python script with a loop to run it for our selected time ranges. The file is called spectrum-extractor.py and we simply run it to get the spectra for the three distinct phases of interest.

In [ ]:
os.chdir(notebook_dir)

print(f"Changed working directory to: {os.getcwd()}")

In [ ]:
os.system(f"python3 {notebook_dir}/scripts/spectrum-extractor.py")

From the final list printed above we can see the names of the grouped spectrum files, we list them below:

In [ ]:
phase1 = f'{wdir}/PN_spectrum_grp_673310879.9999998_32-36_40-44.fits'
phase2 = f'{wdir}/PN_spectrum_grp_673367039.9999999_32-36_40-44.fits'
phase3 = f'{wdir}/PN_spectrum_grp_673382591.9999999_32-36_40-44.fits'

Now we can visualise them using pyxspec.

In [ ]:
# Before you start, clear all data and models

AllModels.clear()  # Clear all models
AllData.clear()    # Clear all data

In [ ]:
# Load the group spectral file

# The group file already contains in the header the names of all the necessary files needed by xspec
# (source and background spectra, response and ancillery files)

data1 = Spectrum(phase1)  # load spectra groupped file
data2 = Spectrum(phase2)
data3 = Spectrum(phase3)

AllData.show()                # inspect loaded data

For swift visualisation, we can use the the custom made Python function available in the file xspecplot.py. It will require a list of the spectrum data given to xspec only, the rest can be changed if customisation is required by the user:
```
xspecplot(data, xAxis="keV", xLog=True, yLog=True, figname="multiple-spectra.png")
```

In [ ]:
xspecplot([data1, data2, data3], xAxis="keV", xLog=True, yLog=True)

* ### Energy-resolved light curves

At the spectra produced above, 4 energy bands of interest have been marked with the vertical red dashed line:
* 0.5-3.0 keV
* 3.0-6.0 keV
* 6.0-8.0 keV
* 8.0-10.0 keV

We can extract light curves for the each band of interest for further analysis of variability in photon count rates. Instead of running all the tasks one by one as in the previous section named `Extracting Average Light Curve`, we have written a Python script to automatise the process and run it for our selected ranges. The script is present underthe name `energy-resolvedLC.py`, which we run below: 

In [ ]:
os.system(f"python3 {notebook_dir}/scripts/energy-resolvedLC.py")

After the script has finished we have out corrected light curves printed out, we have written them below:

In [ ]:
band1 = f'{wdir}/PN_lccorr_500to3000eV_bin283sec.lc'
band2 = f'{wdir}/PN_lccorr_3000to6000eV_bin283sec.lc'
band3 = f'{wdir}/PN_lccorr_6000to8000eV_bin283sec.lc'
band4 = f'{wdir}/PN_lccorr_8000to10000eV_bin283sec.lc'

Now we can plot all the energy-resolved light curves.

In [ ]:
# Inspect the light curves
fileNames = [band1, band2, band3, band4]
names = ['0.5-3.0 keV', '3.0-6.0 keV', '6.0-8.0 keV', '8.0-10.0 keV']

plotVelaX1LC(fileNames, names, threshold=None, figname="energyresolved_lightcurves.png", yLog=True, connect_points=False)

* ### Pulse by pulse analysis

If the user would like to explore further in depth analysis to observe variability it is possible to do a pulse-by-pulse analysis. We take the pulse of Vela X-1 to be $P= 283.44$ s in accordance with Diez et al. (2022). 

We can use the SAS task `tabgtigen` to select GTI files for each pulse period throughout the observation. Below we run a custom script written in Python which should create all the GTI files by running a loop.

In [ ]:
os.system(f"python3 {notebook_dir}/scripts/gtiloop.py")

Now we can produce spectra for the each GTI file for further analysis using the second custom python script, which will run a loop.

In [ ]:
os.system(f"python3 {notebook_dir}/scripts/loopgtispectra.py")

After this point it is up to the user to do further spectral analysis.

***
### Conclusions

Congrats!✨ 

By following this thread you have:

* processed ODF files for the X-ray binary Vela X-1.
* applied calibration in timing mode.
* produced lightcurves of the source and the background.
* checked and corrected for plie-up.
* produced an average spectrum of the object and plotted it using pyXspec.
* produced for different observation phases
* produced a spectrum for each pulse period of the binary

This shows how a further analysis can be done on an object observed by XMM-Newton in the XMM-Newton-SAS Datalab, where you can swiftly get your data and analyse it with all the major tools necessary available at your disposal. The further analysis and model fitting of the object can be advanced by the reader, by exploring the tools at disposal by pyXspec, for more information please refer to the [pyXspec information page](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html).